In [17]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from compare_paths import *
from utils import *
from pytorch_value_iteration_networks.model import *
from types import SimpleNamespace
from eval import *
import pickle
import pandas as pd 

# VIN path compare

In [18]:

#df = pd.read_csv("path_results.csv")
df = pd.read_csv("/Users/nathankeplinger/Documents/Vanderbilt/Research/fullyObservableNavigation/vin_replan.csv")

In [19]:
df.head()

,seed,algorithm,mean_inference_time,total_steps,successful,path,stop_reason,path_vin,path_replan
0,13400,VI,0.022136,124,True,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",Goal Reached,NaN,NaN
1,13400,VIN,0.007765,134,True,NaN,Goal Reached,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",NaN
2,13400,VIN_REPLAN,0.007007,157,True,NaN,Goal Reached,NaN,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0..."
3,6161,VI,0.021535,58,True,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",Goal Reached,NaN,NaN
4,6161,VIN,0.007386,58,True,NaN,Goal Reached,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",NaN


In [20]:
len(df[df["algorithm"] == "VIN"])

279

In [21]:
success_rate_vin = df[df["algorithm"] == "VIN"]["successful"].mean()
success_rate_replan = df[df["algorithm"] == "VIN_REPLAN"]["successful"].mean()
success_rate_vi = df[df["algorithm"] == "VI"]["successful"].mean()


In [23]:
print("Success rate for VIN: ", success_rate_vin)
print("Success rate for VIN Replan: ", success_rate_replan)
print("Success rate for VI: ", success_rate_vi)

Success rate for VIN:  0.8817204301075269
Success rate for VIN Replan:  0.967741935483871
Success rate for VI:  1.0


In [25]:
path_len_vin = df[df["algorithm"] == "VIN"]["total_steps"].mean()
path_len_puct = df[df["algorithm"] == "VIN_REPLAN"]["total_steps"].mean()
path_len_vi = df[df["algorithm"] == "VI"]["total_steps"].mean()

print("Path length for VIN: ", path_len_vin)
print("Path length for Replan: ", path_len_puct)
print("Path length for VI: ", path_len_vi)


Path length for VIN:  88.66308243727599
Path length for Replan:  61.18996415770609
Path length for VI:  47.831541218637994


In [27]:
# # vi_df = df[df["algorithm"]=="VI"]
# # vi_fails = vi_df[vi_df["successful"] == False]
# fail_seeds = df[df["successful"] == False]
# fail_seeds[fail_seeds["seed"]==16515]["path_replan"].iloc[1]

In [28]:
puct_success = df[(df["algorithm"] == "VIN") & (df["successful"] == False)] 

puct_success.head()


,seed,algorithm,mean_inference_time,total_steps,successful,path,stop_reason,path_vin,path_replan
25,9241,VIN,0.007426,39,False,NaN,collision,"[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2...",NaN
58,14030,VIN,0.007354,501,False,NaN,Max Steps Reached,"[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2...",NaN
67,15521,VIN,0.007383,501,False,NaN,Max Steps Reached,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",NaN
88,18040,VIN,0.007512,41,False,NaN,collision,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",NaN
109,13029,VIN,0.007405,501,False,NaN,Max Steps Reached,"[(0, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 0...",NaN


In [ ]:
puct_fail_seeds_paths = df[df["seed"].isin(fail_seeds)]


paths = puct_fail_seeds_paths[puct_fail_seeds_paths["algorithm"] == "VIN_REPLAN"]["path_replan"]

for p in paths:
    print(p)



[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (12, 1), (12, 2), (13, 2), (14, 2), (14, 3), (15, 3), (15, 4), (15, 5), (15, 6), (15, 7), (14, 7), (13, 7), (13, 6), (13, 5), (13, 4), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12, 9), (12, 10), (12, 11), (12, 12), (13, 12), (13, 13), (12, 13), (11, 13), (11, 14), (11, 15), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19), (9, 19), (9, 18), (9, 17), (9, 16), (8, 16), (7, 16), (7, 17), (7, 18), (7, 19), (6, 19), (6, 18), (6, 17), (5, 17), (5, 16), (6, 16), (6, 15), (7, 15), (8, 15), (9, 15), (10, 15), (11, 15), (12, 15), (13, 15), (14, 15), (15, 15), (16, 15), (16, 14), (17, 14), (17, 13), (17, 12), (17, 11), (16, 11), (16, 10), (17, 10), (17, 9), (17, 8), (17, 7), (17, 6), (17, 5), (17, 4), (17, 3), (16, 3), (16, 4), (16, 5), (16, 6), (16, 7), (16, 8), (16, 9), (16, 10), (15, 10), (14, 10), (14, 11), (14, 12), (14, 13), (15, 13), (16, 13), (16, 14), (15, 14), (14, 14), (13, 1

In [29]:
fails = df[df["successful"]==False]

vin_fails = fails[fails["algorithm"]=="VI"]

print(vin_fails["stop_reason"].value_counts())

# print(fails["stop_reason"].value_counts())



Series([], Name: count, dtype: int64)


In [30]:
print(fails[fails["stop_reason"]=="Max Steps Reached"])



      seed algorithm  mean_inference_time  total_steps  successful path  \
58   14030       VIN             0.007354          501       False  NaN   
67   15521       VIN             0.007383          501       False  NaN   
109  13029       VIN             0.007405          501       False  NaN   
121   6149       VIN             0.007432          501       False  NaN   
166   9758       VIN             0.007460          501       False  NaN   
196  15360       VIN             0.007494          501       False  NaN   
262  19271       VIN             0.007500          501       False  NaN   
310  16194       VIN             0.007579          501       False  NaN   
316  14186       VIN             0.007797          501       False  NaN   
325  18660       VIN             0.007472          501       False  NaN   
358  12734       VIN             0.007766          501       False  NaN   
364  12341       VIN             0.007472          501       False  NaN   
376  15766       VIN     

In [ ]:
successes = df[df["successful"]==True]

In [ ]:
successes.sample(10)["seed"].to_list()

[14930, 7316, 10462, 15441, 13724, 12637, 16991, 18213, 13883, 15935]

In [ ]:
vins = df[df["algorithm"]=="VIN"]

print(vins["total_steps"].mean())


45.55670103092783


In [31]:
vis = df[df["algorithm"]=="VIN"]
print(vis["successful"].mean())
print(vis["mean_inference_time"].mean())
print(vis["total_steps"].mean())

0.8817204301075269
0.007623075776662794
88.66308243727599


In [ ]:
# plot steps vs reward collected 


